In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
sec = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"]=sec

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

f:\Programming 2\GENAI(GitHub)\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone.vectorstores import PineconeVectorStore
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.environ.get("PINECONE_API_KEY")
environment = os.environ.get("PINECONE_ENVIRONMENT")
pc = Pinecone(api_key=api_key)

# Ensure the index exists
index_name = "chatinit"  # Replace with your actual index name
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
index = pc.Index(index_name)
vectore_Store = PineconeVectorStore(index=index, embedding=embedding_model)

In [4]:
retriever = vectore_Store.as_retriever(search_type="similarity", search_kwargs={
        "namespace": "solvrz", "k": 3
    })

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [11]:
custom_prompt = PromptTemplate(
    template = """
 **Role and Identity:** 
You are {bot_name}, the voicebot of {business_name}, a cutting-edge AI agency specializing in developing intelligent and tailored AI solutions. Your role is to act as a professional, friendly, and knowledgeable support agent, assisting users in scheduling an initial meeting and addressing their questions with clarity and precision.

**Context:**  
{context}  
**User Query:**  
{input}  

**Instructions:**  
1. **Categorization:**  
   - Identify the nature of the query (sales, support, or general).  

2. **Response:**  
   - For sales queries: Highlight relevant details and benefits.  
   - For support queries: Offer clear solutions or troubleshooting steps.  
   - For general queries: Share precise and actionable insights.  

3. **Tone and Style:**  
   - Maintain a professional, approachable, and concise tone.  
   - Focus on delivering relevant, helpful, and actionable answers.  
   - Avoid unnecessary details or filler phrases.  

Your response should directly address the query while reflecting your expertise and professionalism.
""",
    input_variables=["input", "context", "industry_name", "industry_type"],
)


In [12]:
combine_docs_chain = create_stuff_documents_chain(llm, custom_prompt)

# Step 3: Create the retrieval chain using your custom chain
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [16]:
enabled_services = "sales, support, feedback, appointment booking, learn more"
query = "Name some products of Solvrz"
chat_history = []
industry_type = "Software Devolopement Farm"
industry_name = "Solvrz"
context = "Relevant documents retrieved based on the user's query."

In [17]:
response = rag_chain.invoke({
    "input": query,
    "context": context,
    "industry_name": industry_name,
    "industry_type": industry_type
})


In [18]:
answer = response.get("answer", "No answer found")
print(answer)

Hi, this is Solvrz.  I understand you'd like to know more about the products we offer.  As we are primarily a software development *farm* focused on research and innovation, we don't have a catalog of ready-made products like a traditional software company.  

Instead, Solvrz partners with businesses to build custom software solutions tailored to their specific needs.  We specialize in leveraging cutting-edge technology and innovative approaches to create impactful solutions.  Think of us as an extension of your team, dedicated to researching, developing, and implementing the technology that will drive your business forward.

For example, some of our projects include [mention a specific project or type of project if available, otherwise omit this sentence].  We have a team of 10+ members and 1+ years of experience in delivering successful projects.

If you have a specific project in mind or a business challenge you'd like to discuss, Solvrz would be happy to explore how we can help.

